In [0]:
!pip install optuna

In [0]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras import layers, models, losses, callbacks, regularizers
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras import backend as K
# import os

# # Ensure reproducibility
# np.random.seed(42)
# tf.random.set_seed(42)

# # Check GPU availability
# device = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"

# # Example input (X) and target (y) for demonstration purposes
# n_samples = 16000
# n_features = 1641
# X = np.random.rand(n_samples, n_features).astype(np.float32)
# y = np.random.rand(n_samples).astype(np.float32)

# # Standardize features and target
# scaler_X = StandardScaler()
# scaler_y = StandardScaler()
# X = scaler_X.fit_transform(X)
# y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# # Parameters
# batch_size = 256
# latent_dim = 50
# n_splits = 5

# def build_shallow_autoencoder(input_dim):
#     inputs = layers.Input(shape=(input_dim,))
#     encoded = layers.Dense(512, activation='relu')(inputs)
#     latent = layers.Dense(latent_dim, activation='relu')(encoded)
#     decoded = layers.Dense(512, activation='relu')(latent)
#     outputs = layers.Dense(input_dim)(decoded)
#     autoencoder = models.Model(inputs, outputs)
#     encoder = models.Model(inputs, latent)
#     return autoencoder, encoder

# def build_deep_autoencoder(input_dim):
#     inputs = layers.Input(shape=(input_dim,))
#     encoded = layers.Dense(1024, activation='relu')(inputs)
#     encoded = layers.Dense(512, activation='relu')(encoded)
#     latent = layers.Dense(latent_dim, activation='relu')(encoded)
#     decoded = layers.Dense(512, activation='relu')(latent)
#     decoded = layers.Dense(1024, activation='relu')(decoded)
#     outputs = layers.Dense(input_dim)(decoded)
#     autoencoder = models.Model(inputs, outputs)
#     encoder = models.Model(inputs, latent)
#     return autoencoder, encoder

# def build_variational_autoencoder(input_dim):
#     inputs = layers.Input(shape=(input_dim,))
#     encoded = layers.Dense(512, activation='relu')(inputs)
#     z_mean = layers.Dense(latent_dim)(encoded)
#     z_log_var = layers.Dense(latent_dim)(encoded)

#     def sampling(args):
#         z_mean, z_log_var = args
#         epsilon = tf.random.normal(shape=tf.shape(z_mean))
#         return z_mean + tf.exp(0.5 * z_log_var) * epsilon

#     z = layers.Lambda(sampling)([z_mean, z_log_var])
#     decoder_input = layers.Dense(512, activation='relu')(z)
#     outputs = layers.Dense(input_dim)(decoder_input)

#     vae = models.Model(inputs, outputs)
#     kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
#     vae.add_loss(kl_loss)
#     return vae, models.Model(inputs, z_mean)

# def build_transformer(input_dim):
#     inputs = layers.Input(shape=(input_dim,))
#     x = layers.Reshape((input_dim, 1))(inputs)
#     x = layers.MultiHeadAttention(num_heads=8, key_dim=4)(x, x)
#     x = layers.GlobalAveragePooling1D()(x)
#     latent = layers.Dense(latent_dim, activation='relu')(x)
#     outputs = layers.Dense(1)(latent)
#     transformer = models.Model(inputs, outputs)
#     return transformer

# def train_and_evaluate_model(model_builder, X, y, model_name, kfold):
#     results = []
#     latent_representations = []
#     os.makedirs(model_name, exist_ok=True)

#     for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
#         print(f"Training fold {fold + 1} of {n_splits} for {model_name}")
#         X_train, X_val = X[train_idx], X[val_idx]
#         y_train, y_val = y[train_idx], y[val_idx]

#         with tf.device(device):
#             model, encoder = model_builder(X.shape[1])
#             model.compile(optimizer='adam', loss=losses.MeanSquaredError())
#             es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
#             csv_logger = callbacks.CSVLogger(os.path.join(model_name, f"fold_{fold + 1}_log.csv"))

#             history = model.fit(
#                 X_train, X_train if "autoencoder" in model_name.lower() else y_train,
#                 validation_data=(X_val, X_val if "autoencoder" in model_name.lower() else y_val),
#                 epochs=100,
#                 batch_size=batch_size,
#                 callbacks=[es, csv_logger],
#                 verbose=1
#             )

#             # Save model and latent representations
#             model.save(os.path.join(model_name, f"fold_{fold + 1}.h5"))
#             if "autoencoder" in model_name.lower():
#                 latent_representations.append(encoder.predict(X))
#             else:
#                 results.append(model.evaluate(X_val, y_val, verbose=0))

#     return results, latent_representations

# # K-Fold cross-validator
# kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Train models
# shallow_results, shallow_latents = train_and_evaluate_model(build_shallow_autoencoder, X, y, "Shallow_Autoencoder", kfold)
# deep_results, deep_latents = train_and_evaluate_model(build_deep_autoencoder, X, y, "Deep_Autoencoder", kfold)
# vae_results, vae_latents = train_and_evaluate_model(build_variational_autoencoder, X, y, "Variational_Autoencoder", kfold)
# transformer_results, _ = train_and_evaluate_model(build_transformer, X, y, "Transformer", kfold)

# print("Training complete.")


In [0]:
from ae_models import *

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

In [0]:
def vae_loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD


In [0]:
def train_autoencoder(model, dataloader, num_epochs=100, learning_rate=1e-3):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for data, _ in dataloader:
            optimizer.zero_grad()
            recon_data = model(data)
            loss = criterion(recon_data, data)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
        
        print(f'Epoch {epoch + 1}, Loss: {train_loss / len(dataloader.dataset)}')


model.eval()
with torch.no_grad():
    z_means = []
    for data, _ in dataloader:
        mu, logvar = model.encode(data)
        z_means.append(mu)
    z_means = torch.cat(z_means, dim=0)


In [0]:
def train_autoencoder(model, dataloader, num_epochs=100, learning_rate=1e-3, loss_function=nn.MSELoss()):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    early_stopping = EarlyStopping(patience=10, verbose=True)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for data, _ in dataloader:
            data = data.to(device)
            optimizer.zero_grad()
            if isinstance(model, VAE):
                recon_batch, mu, logvar = model(data)
                loss = vae_loss_function(recon_batch, data, mu, logvar)
            else:
                recon_batch = model(data)
                loss = loss_function(recon_batch, data)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
        
        val_loss = train_loss / len(dataloader.dataset)
        print(f'Epoch {epoch + 1}, Loss: {val_loss}')
        
        early_stopping(val_loss)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
    
    model.eval()
    with torch.no_grad():
        z_means = []
        for data, _ in dataloader:
            mu, logvar = model.encode(data)
            z_means.append(mu)
        z_means = torch.cat(z_means, dim=0)


In [0]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss)
        elif val_loss > self.best_loss:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss)
            self.counter = 0

    def save_checkpoint(self, val_loss):
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'best_autoencoder_model.pth')

# load data

In [0]:
import pandas as pd

y = pd.read_csv("../data/age.csv", header=None)
X = pd.read_csv("../data/processed_log_abundance.csv", header=None, index_col=0)

display(y)
display(X)

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

x_train = x_train.to(device)
y_train = y_train.to(device)

In [0]:
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Train Deep Autoencoder
deep_ae = DeepAE(input_dim).to(device)
train_autoencoder(deep_ae, dataloader)

# Train Shallow Autoencoder
shallow_ae = ShallowAE(input_dim).to(device)
train_autoencoder(shallow_ae, dataloader)

# Train VAE
vae = VAE(input_dim, hidden_dim=512, latent_dim=64).to(device)
train_autoencoder(vae, dataloader, loss_function=vae_loss_function)

# Train Convolutional Autoencoder (reshape data as needed)
# conv_ae = ConvAE().to(device)
# train_autoencoder(conv_ae, dataloader)

In [0]:
def extract_latent_representations(model, dataloader):
    model.load_state_dict(torch.load('best_autoencoder_model.pth'))
    model.eval()
    with torch.no_grad():
        z_means = []
        for data, _ in dataloader:
            if isinstance(model, VAE):
                mu, logvar = model.encode(data)
                z_means.append(mu)
            else:
                z = model.encoder(data)
                z_means.append(z)
        z_means = torch.cat(z_means, dim=0)
    return z_means

# Example usage
z_means = extract_latent_representations(deep_ae, dataloader)

In [0]:
# Convert to numpy arrays for sklearn
z_means_np = z_means.numpy()
y_train_np = y_train.numpy()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(z_means_np, y_train_np, test_size=0.2, random_state=42)

# Train the regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predict and evaluate
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')